In [1]:
''' Outdated models '''
# MODEL_NAME='lucyd-edof.pth'
# MODEL_NAME='lucyd-edof-11.pth'

''' New/relevant models '''
# MODEL_NAME='lucyd-small-phantoms.pth' # original model trained on small phantoms
#MODEL_NAME='lucyd-psf-sim-2.pth' # improved EDOF blur simlation (3d projections + microscopy PSF simulation)
MODEL_NAME='lucyd-edof-plankton_231204.pth' # improved EDOF blur simlation (slightly less patchy results, trained with finer structures)

In [2]:
from lucyd import LUCYD, device
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import torch
import time 
import os

import cv2 as cv
import numpy as np

torch.cuda.set_per_process_memory_fraction(0.9, 0)
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

Parameters:

In [3]:
RESIZE=False
RESIZE_RESOLUTION=(2560,2560) # will be ignored if RESIZE=False
BG_CORR = True

model = LUCYD(num_res=1).to(device)
model.load_state_dict(torch.load('models/'+MODEL_NAME))
model.eval()
print('number of parameters: {}'.format(sum(p.numel() for p in model.parameters())))

path = '/mnt/m181 PISCO1/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/PNG/'

number of parameters: 8452


Background correction:

In [ ]:
if BG_CORR:
    files = os.listdir(path)
    imgs = [cv.imread(os.path.join(path, file), cv.IMREAD_GRAYSCALE) for file in files]
    bg = np.max(imgs, axis=0)
    corrected = cv.absdiff(imgs[0], bg)
else:
    pass

In [4]:
####Dataset 
import torchvision.transforms as transforms
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
from torch.nn.functional import pad
import os
from PIL import Image
import torch.nn.functional as F

class UnlabelledImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.img_names = sorted(os.listdir(img_dir))
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_path).convert('L')
        name = self.img_names[idx]
        if self.transform:
            image = self.transform(image)
        return image, name  # Return a dummy label

def collate_fn(batch):
    # Find the max height and width in this batch
    max_height = max(img.shape[1] for img, _ in batch)
    max_width = max(img.shape[2] for img, _ in batch)

    # Ensure max_height and max_width are even numbers
    max_height += max_height % 2
    max_width += max_width % 2

    # Add padding to images in this batch and collate them into a single tensor
    batch = [F.pad(img, (0, max_width - img.shape[2], 0, max_height - img.shape[1]), mode='constant', value=0) for img, _ in batch]
    return torch.stack(batch), _ 

transform = transforms.Compose([
        #transforms.Resize((2560,2560)),
        #transforms.RandomCrop((512,512)),
        #transforms.Resize((5120,5120)),
        #transforms.RandomResizedCrop(224),
        #transforms.RandomHorizontalFlip(),
        #transforms.RandomRotation(degrees=(0, 180)),
        #transforms.Grayscale(),
        transforms.ToTensor(),
        #transforms.Normalize([0.5,], [0.5,])
    ])

# Load the unseen dataset
dataset = UnlabelledImageDataset('/home/plankton/Data/M181_test_set2/results', transform=transform)

# Create a data loader
#images = DataLoader(dataset, batch_size=1, collate_fn=collate_fn)
images = DataLoader(dataset, batch_size=7)

In [5]:
print(torch.cuda.memory_allocated())
torch.cuda.empty_cache()
print(torch.cuda.memory_reserved())

119296
2097152


In [7]:
################
import sys
from tqdm import tqdm
# NAME OF THE IMAGE YOU WANT TO DECONVOLVE
#path = '/mnt/m181 PISCO1/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/PNG/'
index = 0
with torch.no_grad():
    for image, name in iter(images):
        start_time = time.time()
        print(name)

        #filename = os.path.join(path,file)
        #result_fn = os.path.join('/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543',file)
        result_fn = os.path.join('/home/plankton/Data/M181_test_set2/deconv_test', str(name)+'.png')
        index += 1 

        ################

            #if RESIZE:
            #    x = np.array(Image.open(filename).convert('L').resize(RESIZE_RESOLUTION))
            #else:
            #    x = np.array(Image.open(filename).convert('L'))
        x = image
        

            #if x.shape[0] % 2 != 0: x = x[1:]
            #if x.shape[1] % 2 != 0: x = x[:,1:]
            # x = (x - np.min(x))/(np.max(x) - np.min(x))
        

        x_torch = x.to(device)
        #print(torch.cuda.memory_allocated())
        #print(torch.cuda.memory_reserved())
        #torch.cuda.synchronize()        
            #x_torch = x_torch.unsqueeze(0).unsqueeze(0)
        print('Time to load image: {} seconds'.format(time.time() - start_time))
        start_time = time.time()
        y_hat, y_k, update = model(x_torch.float())
        torch.cuda.synchronize()
        print('Time to deconvolve: {} seconds'.format(time.time() - start_time))
        start_time = time.time()
            # plt.figure(figsize=(20,20))
            # plt.subplot(1,2,1)
        #y = x_torch.detach().cpu().numpy()[0,0]
        #print('Time to detach y image: {} seconds'.format(time.time() - start_time))
            # plt.imshow(y, cmap='gray')
            # plt.title('Input')
            # plt.subplot(1,2,2)
        x = y_hat.detach().cpu().numpy()[0,0]
        print('Time to cpu x image: {} seconds'.format(time.time() - start_time))
        #x = x.numpy()[0,0]
        #print('Time to numpy x image: {} seconds'.format(time.time() - start_time))
        start_time = time.time()

            # plt.imshow(x, cmap='gray', vmin=0)
            # plt.title('Output')
            # plt.show()
        x = x*255*1.1
        #cv.imwrite(result_fn, x)
        del x

        #Image.fromarray(x).convert('L').save(result_fn)
        print('Time to save image: {} seconds'.format(time.time() - start_time))
        #torch.cuda.empty_cache()



('inverted_0.png', 'inverted_1.png', 'inverted_10.png', 'inverted_11.png', 'inverted_12.png', 'inverted_13.png', 'inverted_14.png')
Time to load image: 0.037109375 seconds
Time to deconvolve: 1.0152018070220947 seconds
Time to cpu x image: 0.028794050216674805 seconds
Time to save image: 0.00475311279296875 seconds
('inverted_15.png', 'inverted_16.png', 'inverted_17.png', 'inverted_18.png', 'inverted_19.png', 'inverted_2.png', 'inverted_20.png')
Time to load image: 0.03096628189086914 seconds
Time to deconvolve: 1.013456106185913 seconds
Time to cpu x image: 0.1009521484375 seconds
Time to save image: 0.013643264770507812 seconds
('inverted_21.png', 'inverted_22.png', 'inverted_23.png', 'inverted_24.png', 'inverted_25.png', 'inverted_26.png', 'inverted_27.png')
Time to load image: 0.030823707580566406 seconds
Time to deconvolve: 1.0142197608947754 seconds
Time to cpu x image: 0.10175919532775879 seconds
Time to save image: 0.01735210418701172 seconds
('inverted_28.png', 'inverted_29.pn

## Batchsize = 1
allocated: 104976896
reserved: 1155530752
Time to load image: 0.003166675567626953 seconds
Time to deconvolve: 0.19535279273986816 seconds
Time to cpu x image: 0.0033130645751953125 seconds
Time to save image: 0.024829626083374023 seconds

Time per image ~0.2257 seconds

## Batchsize = 4 
allocated: 419549696
reserved: 1260388352
Time to load image: 0.017978191375732422 seconds
Time to deconvolve: 0.5801444053649902 seconds
Time to cpu x image: 0.016628026962280273 seconds
Time to save image: 0.023236751556396484 seconds

Time per image ~0.1615 seconds

## Batchsize = 7
734122496
2311061504
Time to load image: 0.031010866165161133 seconds
Time to deconvolve: 1.0197343826293945 seconds
Time to cpu x image: 0.10259056091308594 seconds
Time to save image: 0.03341197967529297 seconds

Time per image ~0.1571 seconds


In [ ]:
index = 0
with torch.no_grad():
    for image, _ in images:
        start_time = time.time()

        result_fn = os.path.join('/home/plankton/Results/M181/Deconv/batch_test',str(index)+'.png')
        index += 1 

        x = image

        x_torch = x.to(device)
        print('Time to load image: {} seconds'.format(time.time() - start_time))
        start_time = time.time()
        y_hat, y_k, update = model(x_torch.float())
        print('Time to deconvolve: {} seconds'.format(time.time() - start_time))
        start_time = time.time()
            # plt.figure(figsize=(20,20))
            # plt.subplot(1,2,1)
        #y = x_torch.detach().cpu().numpy()[0,0]
        #print('Time to detach y image: {} seconds'.format(time.time() - start_time))
            # plt.imshow(y, cmap='gray')
            # plt.title('Input')
            # plt.subplot(1,2,2)
        x = y_hat.detach().cpu().numpy()[0,0]
        #print('Time to detach x image: {} seconds'.format(time.time() - start_time))
            # plt.imshow(x, cmap='gray', vmin=0)
            # plt.title('Output')
            # plt.show()
        x = x*255
        cv.imwrite(result_fn, x)

        #Image.fromarray(x).convert('L').save(result_fn)
        print('Time to save image: {} seconds'.format(time.time() - start_time))

In [27]:
#!cd /home/plankton/PISCO_Software/MaxSegmenter
from MaxSegmenterModule import run_segmenter

run_segmenter(
        #source_folder="C:/Users/timka/Documents/Arbeit/PeruTest",
        source_folder="/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/",
        save_crops=False,
        save_marked_imgs=False,
        save_bg_corr_imgs=True,
        min_area_to_segment=1000,
        min_area_to_save=1000,
        save_path="/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/bg_corr/",
        #save_path="C:/Users/timka/Documents/Arbeit/Results",
        equalize_hist=True,
        resize=True,
        clear_save_path=True,
        bg_size=5,
        max_threads=10,
        n_sigma=2, #2 for M181 images, 1.5 for Cusco Peru
        n_cores=8,
        mask_imgs=True,
        mask_radius_offset=200,
    )

unprocessed directory found!
remaining images: 20522
Start segmenting
Mask radius is: 1302
10% done
30% done
40% done
50% done
70% done
80% done
100% done
Total time: 1610.5983849840704, Avg time per img: 0.07848155077400207


In [26]:
%cd /home/plankton/PISCO_Software/MaxSegmenter
%pwd

/home/plankton/PISCO_Software/MaxSegmenter


'/home/plankton/PISCO_Software/MaxSegmenter'

In [28]:
run_segmenter(
        #source_folder="C:/Users/timka/Documents/Arbeit/PeruTest",
        source_folder="/mnt/m181 PISCO1/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/PNG/",
        save_crops=False,
        save_marked_imgs=False,
        save_bg_corr_imgs=True,
        min_area_to_segment=1000,
        min_area_to_save=1000,
        save_path="/home/plankton/Results/M181/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/",
        #save_path="C:/Users/timka/Documents/Arbeit/Results",
        equalize_hist=True,
        resize=True,
        clear_save_path=True,
        bg_size=5,
        max_threads=10,
        n_sigma=2, #2 for M181 images, 1.5 for Cusco Peru
        n_cores=8,
        mask_imgs=True,
        mask_radius_offset=200,
    )

unprocessed directory found!
remaining images: 20521
Start segmenting
Mask radius is: 2820
No progress detected, terminating status update...
Total time: 2011.0037388849305, Avg time per img: 0.09799735582500514


In [14]:
################
from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)

# NAME OF THE IMAGE YOU WANT TO DECONVOLVE
path = '/home/plankton/Data/M181_test_set2'
for file in os.listdir(path):

    filename = os.path.join(path,file)
    result_fn = os.path.join('/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543_bg_corr',file)
    ################

    with torch.no_grad():
        if RESIZE:
            x = np.array(Image.open(filename).convert('L').resize(RESIZE_RESOLUTION))
        else:
            x = np.array(Image.open(filename).convert('L'))

        if x.shape[0] % 2 != 0: x = x[1:]
        if x.shape[1] % 2 != 0: x = x[:,1:]
        # x = (x - np.min(x))/(np.max(x) - np.min(x))
        x = x/255

        x_torch = torch.from_numpy(x).to(device)
        #y_torch = torch.from_numpy(x).to(device)

        x_torch = x_torch.unsqueeze(0).unsqueeze(0)

        start_time = time.time()
        y_hat, y_k, update = model(x_torch.float())
        print('Time to deconvolve: {} seconds'.format(time.time() - start_time))

        # plt.figure(figsize=(20,20))
        # plt.subplot(1,2,1)
        #y = x_torch.detach().cpu().numpy()[0,0]
        # plt.imshow(y, cmap='gray')
        # plt.title('Input')
        # plt.subplot(1,2,2)
        x = y_hat.detach().cpu().numpy()[0,0]
        # # plt.imshow(x, cmap='gray', vmin=0)
        # # plt.title('Output')
        # # plt.show()
        info = nvmlDeviceGetMemoryInfo(h)


        print(f'total   : {info.total * 1e-6} MB')
        print(f'free   : {info.free* 1e-6} MB')
        print(f'used   :  {info.used* 1e-6} MB')
        print(torch.cuda.memory_allocated(0) * 1e-6)
        print(torch.cuda.memory_reserved(0) * 1e-6)

        torch.cuda.empty_cache()




        Image.fromarray(x*255.).convert('L').save(result_fn)

OutOfMemoryError: CUDA out of memory. Tried to allocate 400.00 MiB (GPU 0; 7.78 GiB total capacity; 6.36 GiB already allocated; 707.19 MiB free; 7.01 GiB allowed; 6.64 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [15]:
info = nvmlDeviceGetMemoryInfo(h)


print(f'total   : {info.total * 1e-6} MB')
print(f'free   : {info.free* 1e-6} MB')
print(f'used   :  {info.used* 1e-6} MB')
print(torch.cuda.memory_allocated(0) * 1e-6)
print(torch.cuda.memory_reserved(0) * 1e-6)

total   : 8589.934592 MB
free   : 741.5398399999999 MB
used   :  7848.394751999999 MB
6824.89856
7132.413952
